Implementation of CycleGAN <br>
Project page: https://junyanz.github.io/CycleGAN/ <br>
Step: <br>
1. Download dataset from https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/
2. Make new folder named 'data'
3. Put the dataset into data folder
4. Modify the hyperparameters and Run the 1st~5th cells in order
5. After (4), you can generate current output by running the 6th cell

In [3]:
from keras.layers import Layer, Input, Conv2D, Activation, add, BatchNormalization, UpSampling2D, ZeroPadding2D, Conv2DTranspose, Flatten, MaxPooling2D, AveragePooling2D,InputSpec
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization


from keras.layers.advanced_activations import LeakyReLU

from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.backend import mean
from keras.models import Model, model_from_json
from keras.utils import plot_model
from keras.engine.topology import Network

from collections import OrderedDict
from scipy.misc import imsave, toimage  
import numpy as np
import random
import datetime
import time
import json
import math
import csv
import sys
import os
from IPython.display import clear_output
from PIL import Image
import keras.backend as K
import tensorflow as tf

np.random.seed(seed=1111)

class CycleGAN():
    def __init__(self, lr_D=2e-4, lr_G=2e-4, image_shape=(256, 256, 3),
                 date_time_string_addition='_test', image_folder='apple2orange'):
        self.img_shape = image_shape
        self.channels = 3
        self.normalization = InstanceNormalization
        # Hyper parameters
        self.lambda_C = 10.0 # weight for Cycle-Consistency loss
        self.lambda_D = 1.0  # Weight for DA,DB
        self.learning_rate_D = lr_D
        self.learning_rate_G = lr_G
        self.generator_iterations = 1 
        self.discriminator_iterations = 1  
        self.beta_1 = 0.5
        self.beta_2 = 0.999
        self.batch_size = 1
        self.epochs = 100 
        self.save_interval = 1
        self.synthetic_pool_size = 50
        #special options : identity learning to avoid opposite output of background (non_ROI regions)
        self.id_learning=False
        self.id_weight=0.5
        
        self.REAL_LABEL = 1.0
        # Used as storage folder name
        self.date_time = time.strftime('%Y%m%d-%H%M%S', time.localtime()) + date_time_string_addition
        
        #optimizer
        self.opt_D = Adam(self.learning_rate_D, self.beta_1, self.beta_2)
        self.opt_G = Adam(self.learning_rate_G, self.beta_1, self.beta_2)
       
        D_A = self.Build_D()
        D_B = self.Build_D()
        loss_weights_D = [0.5]  

        # DA,DB build up
        image_A = Input(shape=self.img_shape)
        image_B = Input(shape=self.img_shape)
        guess_A = D_A(image_A)
        guess_B = D_B(image_B)
        self.D_A = Model(inputs=image_A, outputs=guess_A, name='D_A_model')
        self.D_B = Model(inputs=image_B, outputs=guess_B, name='D_B_model')
        self.D_A.compile(optimizer=self.opt_D,
                         loss=self.lse,
                         loss_weights=loss_weights_D)
        self.D_B.compile(optimizer=self.opt_D,
                         loss=self.lse,
                         loss_weights=loss_weights_D)

        
        self.D_A_static = Network(inputs=image_A, outputs=guess_A, name='D_A_static_model')
        self.D_B_static = Network(inputs=image_B, outputs=guess_B, name='D_B_static_model')
        self.D_A_static.trainable = False
        self.D_B_static.trainable = False
        """
        without static Ds, the Ds in the G_model may be trainable even after setting the trainable var to False
        """

        # Generators build up
        self.G_A2B = self.Build_G(name='G_A2B_model')
        self.G_B2A = self.Build_G(name='G_B2A_model')

        real_A = Input(shape=self.img_shape, name='real_A')
        real_B = Input(shape=self.img_shape, name='real_B')
        synthetic_B = self.G_A2B(real_A)
        synthetic_A = self.G_B2A(real_B)
        dA_guess_synthetic = self.D_A_static(synthetic_A)
        dB_guess_synthetic = self.D_B_static(synthetic_B)
        reconstructed_A = self.G_B2A(synthetic_B)
        reconstructed_B = self.G_A2B(synthetic_A)
        if(self.id_learning):
            identity_A=self.G_A2B(real_A)
            identity_B=self.G_B2A(real_B)
        
        model_outputs = [reconstructed_A, reconstructed_B]
        model_outputs.append(dA_guess_synthetic)
        model_outputs.append(dB_guess_synthetic)
        if(self.id_learning):
            model_outputs.append(identity_A)
            model_outputs.append(identity_B)

            compile_losses = ['mae','mae','mse','mse','mae','mae']
            compile_weights = [self.lambda_C, self.lambda_C,
                               self.lambda_D, self.lambda_D,
                               self.lambda_C*self.id_weight,self.lambda_C*self.id_weight]
        else:
            compile_losses = ['mae','mae','mse','mse']
            compile_weights = [self.lambda_C, self.lambda_C,
                           self.lambda_D, self.lambda_D]

        self.G_model = Model(inputs=[real_A, real_B],
                             outputs=model_outputs,
                             name='G_model')

        self.G_model.compile(optimizer=self.opt_G,
                             loss=compile_losses,
                             loss_weights=compile_weights)
        # self.G_A2B.summary()

        # Data    
        print('--- Caching datasets---')
        # Data Loading
        def load_data(nr_of_channels=3, batch_size=1, subfolder=''):
            trainA_path = os.path.join('data', subfolder, 'trainA')
            trainB_path = os.path.join('data', subfolder, 'trainB')
            testA_path = os.path.join('data', subfolder, 'testA')
            testB_path = os.path.join('data', subfolder, 'testB')
            trainA_image_names=os.listdir(trainA_path)
            trainB_image_names=os.listdir(trainB_path)
            testA_image_names=os.listdir(testA_path)
            testB_image_names=os.listdir(testB_path)
            trainA_images = create_image_array(trainA_image_names, trainA_path, nr_of_channels)
            trainB_images = create_image_array(trainB_image_names, trainB_path, nr_of_channels)
            testA_images = create_image_array(testA_image_names, testA_path, nr_of_channels)
            testB_images = create_image_array(testB_image_names, testB_path, nr_of_channels)
            return {"trainA_images": trainA_images, "trainB_images": trainB_images,
                    "testA_images": testA_images, "testB_images": testB_images,
                    "trainA_image_names": trainA_image_names,
                    "trainB_image_names": trainB_image_names,
                    "testA_image_names": testA_image_names,
                    "testB_image_names": testB_image_names}
        def create_image_array(image_list, image_path, nr_of_channels):
            image_array = []
            for image_name in image_list:
                if image_name[-1].lower() == 'g':  # to avoid e.g. thumbs.db files
                    if nr_of_channels == 1:  # Gray scale image -> MR image
                        image = np.array(Image.open(os.path.join(image_path, image_name)))
                        image = image[:, :, np.newaxis]
                    else:                   # RGB image -> street view
                        image = np.array(Image.open(os.path.join(image_path, image_name)))
                        if(np.array(image).shape==(256,256)):
                            image=np.stack((image,)*3,axis=-1)
                    image = normalize_array(image)
                    image_array.append(image)

            return np.array(image_array)

        # convert 0~255 to -1~1
        def normalize_array(array):
            array = array / 127.5 - 1
            return array


        data =load_data(nr_of_channels=self.channels,
                           batch_size=self.batch_size,
                           subfolder=image_folder)

        self.A_train = data["trainA_images"]
        self.B_train = data["trainB_images"]
        self.A_test = data["testA_images"]
        self.B_test = data["testB_images"]
        self.testA_image_names = data["testA_image_names"]
        self.testB_image_names = data["testB_image_names"]


        # Create run folder and store meta data
        directory = os.path.join('images', self.date_time)
        if not os.path.exists(directory):
            os.makedirs(directory)
        self.writeMetaDataToJSON()

        config = tf.ConfigProto()

        config.gpu_options.allow_growth = True

        K.tensorflow_backend.set_session(tf.Session(config=config))

        sys.stdout.flush()
        
# Layers help functions
    def ck(self, x, k, use_normalization):
        x = Conv2D(filters=k, kernel_size=4, strides=2, padding='same')(x)
        # Normalization is not done on the first discriminator layer
        if use_normalization:
            x = self.normalization(axis=3, center=True, epsilon=1e-5)(x, training=True)
        x = LeakyReLU(alpha=0.2)(x)
        return x
    def c7Ak(self, x, k):
        x = Conv2D(filters=k, kernel_size=7, strides=1, padding='valid')(x)
        x = self.normalization(axis=3, center=True, epsilon=1e-5)(x, training=True)
        x = Activation('relu')(x)
        return x
    def dk(self, x, k):
        x = Conv2D(filters=k, kernel_size=3, strides=2, padding='same')(x)
        x = self.normalization(axis=3, center=True, epsilon=1e-5)(x, training=True)
        x = Activation('relu')(x)
        return x
    def Rk(self, x0):
        k = int(x0.shape[-1])
        # first layer
        x = ReflectionPadding2D((1,1))(x0)
        x = Conv2D(filters=k, kernel_size=3, strides=1, padding='valid')(x)
        x = self.normalization(axis=3, center=True, epsilon=1e-5)(x, training=True)
        x = Activation('relu')(x)
        # second layer
        x = ReflectionPadding2D((1, 1))(x)
        x = Conv2D(filters=k, kernel_size=3, strides=1, padding='valid')(x)
        x = self.normalization(axis=3, center=True, epsilon=1e-5)(x, training=True)
        # merge
        x = add([x, x0])
        return x
    def uk(self, x, k):
        x = Conv2DTranspose(filters=k, kernel_size=3, strides=2, padding='same')(x)  # this matches fractinoally stided with stride 1/2
        x = self.normalization(axis=3, center=True, epsilon=1e-5)(x, training=True)
        x = Activation('relu')(x)
        return x
# Models helper function
    def Build_D(self, name=None):
        # Specify input
        input_img = Input(shape=self.img_shape)
        x = self.ck(input_img, 64, False)
        x = self.ck(x, 128, True)
        x = self.ck(x, 256, True)
        x = self.ck(x, 512, True)      
        x = Conv2D(filters=1, kernel_size=4, strides=1, padding='same')(x)
        x = Activation('sigmoid')(x)
        return Model(inputs=input_img, outputs=x, name=name)
    def Build_G(self, name=None):
        # Specify input
        input_img = Input(shape=self.img_shape)
        # convolution
        x = ReflectionPadding2D((3, 3))(input_img)
        x = self.c7Ak(x, 32)
        x = self.dk(x, 64)
        x = self.dk(x, 128)
        # Residual blocks
        for _ in range(4, 13):
            x = self.Rk(x)
        # up-sampling
        x = self.uk(x, 64)
        x = self.uk(x, 32)
        x = ReflectionPadding2D((3, 3))(x)
        x = Conv2D(self.channels, kernel_size=7, strides=1)(x)
        x = Activation('tanh')(x)  
        return Model(inputs=input_img, outputs=x, name=name)
# Training
    def train(self, epochs, batch_size=1, save_interval=1):
        def run_training_iteration(loop_index, epoch_iterations):
            # Discriminator training 
            synthetic_images_B = self.G_A2B.predict(real_images_A)
            synthetic_images_A = self.G_B2A.predict(real_images_B)
            synthetic_images_A = synthetic_pool_A.query(synthetic_images_A)
            synthetic_images_B = synthetic_pool_B.query(synthetic_images_B)

            for _ in range(self.discriminator_iterations):
                DA_loss_real = self.D_A.train_on_batch(x=real_images_A, y=ones)
                DB_loss_real = self.D_B.train_on_batch(x=real_images_B, y=ones)
                DA_loss_synthetic = self.D_A.train_on_batch(x=synthetic_images_A, y=zeros)
                DB_loss_synthetic = self.D_B.train_on_batch(x=synthetic_images_B, y=zeros)
                
                DA_loss = DA_loss_real + DA_loss_synthetic
                DB_loss = DB_loss_real + DB_loss_synthetic
                D_loss = DA_loss + DB_loss

                if self.discriminator_iterations > 1:
                    print('D_loss:', D_loss)
                    sys.stdout.flush()

            # Generator training 
            target_data = [real_images_A, real_images_B]  # Compare reconstructed images to real images
            
            target_data.append(ones)
            target_data.append(ones)
            if(self.id_learning):
                target_data.append(real_images_A)
                target_data.append(real_images_B)            

            for _ in range(self.generator_iterations):
                G_loss = self.G_model.train_on_batch(
                    x=[real_images_A, real_images_B], y=target_data)
                if self.generator_iterations > 1:
                    print('G_loss:', G_loss)
                    sys.stdout.flush()
            reconstruction_loss_A = G_loss[1]
            reconstruction_loss_B = G_loss[2]
            gA_d_loss_synthetic = G_loss[3]
            gB_d_loss_synthetic = G_loss[4]
            if(self.id_learning):
                identity_loss_A=G_loss[5]
                identity_loss_B=G_loss[6]
            
            # Store training data
            DA_losses.append(DA_loss)
            DB_losses.append(DB_loss)
            gA_d_losses_synthetic.append(gA_d_loss_synthetic)
            gB_d_losses_synthetic.append(gB_d_loss_synthetic)
            gA_losses_reconstructed.append(reconstruction_loss_A)
            gB_losses_reconstructed.append(reconstruction_loss_B)

            GA_loss = gA_d_loss_synthetic + reconstruction_loss_A
            GB_loss = gB_d_loss_synthetic + reconstruction_loss_B
            if(self.id_learning):
                GA_loss+=identity_loss_A
                GB_loss+=identity_loss_B
            D_losses.append(D_loss)
            GA_losses.append(GA_loss)
            GB_losses.append(GB_loss)
            G_losses.append(G_loss)
            reconstruction_loss = reconstruction_loss_A + reconstruction_loss_B
            reconstruction_losses.append(reconstruction_loss)
            clear_output()
            print('\n')
            print('Epoch----------------', epoch, '/', epochs)
            print('Loop index----------------', loop_index + 1, '/', epoch_iterations)
            print('D_loss: ', D_loss)
            print('G_loss: ', G_loss[0])
            print('reconstruction_loss: ', reconstruction_loss)
            print('dA_loss:', DA_loss)
            print('DB_loss:', DB_loss)

            if loop_index % 20 == 0:
                # Save temporary images 
                self.save_tmp_images(real_images_A, real_images_B, synthetic_images_A, synthetic_images_B)
                #self.print_ETA(start_time, epoch, epoch_iterations, loop_index)

        # Begin training
        training_history = OrderedDict()

        DA_losses = []
        DB_losses = []
        gA_d_losses_synthetic = []
        gB_d_losses_synthetic = []
        gA_losses_reconstructed = []
        gB_losses_reconstructed = []

        GA_losses = []
        GB_losses = []
        reconstruction_losses = []
        D_losses = []
        G_losses = []

        # Image pools used to update the discriminators
        synthetic_pool_A = ImagePool(self.synthetic_pool_size)
        synthetic_pool_B = ImagePool(self.synthetic_pool_size)
        
        label_shape = (batch_size,) + self.D_A.output_shape[1:]
        ones = np.ones(shape=label_shape) * self.REAL_LABEL
        zeros = ones * 0

        start_time = time.time()

        for epoch in range(1, epochs + 1):
            A_train = self.A_train
            B_train = self.B_train
            # if train dataset>2000,only train 2000 iteration for each epoch
            epoch_iterations=min(2000,max(len(A_train),len(B_train)))
            random_order_A = np.random.randint(len(A_train), size=epoch_iterations)
            random_order_B = np.random.randint(len(B_train), size=epoch_iterations)
            
            print("epoch: ",epoch," iterations: ",epoch_iterations)


            for loop_index in range(0, epoch_iterations,batch_size):
                index_A=random_order_A[loop_index:loop_index+batch_size]
                index_B=random_order_B[loop_index:loop_index+batch_size]

                sys.stdout.flush()
                real_images_A = A_train[index_A]
                real_images_B = B_train[index_B]

                # Run all training steps
                run_training_iteration(loop_index, epoch_iterations)

            print('\n')
            print('Epoch----------------', epoch, '/', epochs)
            print('D_loss: ',D_losses[-1])
            print('G_loss: ',G_losses[-1][0])
            print('reconstructed loss: ',reconstruction_losses[-1])
            print('DA_loss: ',DA_losses[-1])
            print('DB_loss: ',DB_losses[-1])
            if epoch % save_interval == 0:
                print('\n', '\n', '-------------------------Saving images for epoch', epoch, '-------------------------', '\n', '\n')
                self.saveImages(epoch, real_images_A, real_images_B)

            if epoch % 20 == 0:
                
                self.saveModel(self.D_A, epoch)
                self.saveModel(self.D_B, epoch)
                self.saveModel(self.G_A2B, epoch)
                self.saveModel(self.G_B2A, epoch)

            training_history = {
                'DA_losses': DA_losses,
                'DB_losses': DB_losses,
                'gA_d_losses_synthetic': gA_d_losses_synthetic,
                'gB_d_losses_synthetic': gB_d_losses_synthetic,
                'gA_losses_reconstructed': gA_losses_reconstructed,
                'gB_losses_reconstructed': gB_losses_reconstructed,
                'D_losses': D_losses,
                'G_losses': G_losses,
                'reconstruction_losses': reconstruction_losses}
            self.writeLossDataToFile(training_history)

            sys.stdout.flush()

# other helper functions
    def lse(self, y_true, y_pred):
        loss = tf.reduce_mean(tf.squared_difference(y_pred, y_true))
        return loss
    def truncateAndSave(self, real_, real, synthetic, reconstructed, path_name):
        if len(real.shape) > 3:
            real = real[0]
            synthetic = synthetic[0]
            reconstructed = reconstructed[0]

        # Append and save
        if real_ is not None:
            if len(real_.shape) > 4:
                real_ = real_[0]
            image = np.hstack((real_[0], real, synthetic, reconstructed))
        else:
            image = np.hstack((real, synthetic, reconstructed))

        if self.channels == 1:
            image = image[:, :, 0]

        toimage(image, cmin=-1, cmax=1).save(path_name)

    def saveImages(self, epoch, real_image_A, real_image_B, num_saved_images=1):
        directory = os.path.join('images', self.date_time)
        if not os.path.exists(os.path.join(directory, 'A')):
            os.makedirs(os.path.join(directory, 'A'))
            os.makedirs(os.path.join(directory, 'B'))
            os.makedirs(os.path.join(directory, 'Atest'))
            os.makedirs(os.path.join(directory, 'Btest'))

        testString = ''

        real_image_Ab = None
        real_image_Ba = None
        for i in range(num_saved_images + 1):
            if i == num_saved_images:
                real_image_A = self.A_test[0]
                real_image_B = self.B_test[0]
                real_image_A = np.expand_dims(real_image_A, axis=0)
                real_image_B = np.expand_dims(real_image_B, axis=0)
                testString = 'test'
                
            else:
                #real_image_A = self.A_train[rand_A_idx[i]]
                #real_image_B = self.B_train[rand_B_idx[i]]
                if len(real_image_A.shape) < 4:
                    real_image_A = np.expand_dims(real_image_A, axis=0)
                    real_image_B = np.expand_dims(real_image_B, axis=0)
                
            synthetic_image_B = self.G_A2B.predict(real_image_A)
            synthetic_image_A = self.G_B2A.predict(real_image_B)
            reconstructed_image_A = self.G_B2A.predict(synthetic_image_B)
            reconstructed_image_B = self.G_A2B.predict(synthetic_image_A)

            self.truncateAndSave(real_image_Ab, real_image_A, synthetic_image_B, reconstructed_image_A,
                                 'images/{}/{}/epoch{}_sample{}.png'.format(
                                     self.date_time, 'A' + testString, epoch, i))
            self.truncateAndSave(real_image_Ba, real_image_B, synthetic_image_A, reconstructed_image_B,
                                 'images/{}/{}/epoch{}_sample{}.png'.format(
                                     self.date_time, 'B' + testString, epoch, i))

    def save_tmp_images(self, real_image_A, real_image_B, synthetic_image_A, synthetic_image_B):
        try:
            reconstructed_image_A = self.G_B2A.predict(synthetic_image_B)
            reconstructed_image_B = self.G_A2B.predict(synthetic_image_A)

            real_images = np.vstack((real_image_A[0], real_image_B[0]))
            synthetic_images = np.vstack((synthetic_image_B[0], synthetic_image_A[0]))
            reconstructed_images = np.vstack((reconstructed_image_A[0], reconstructed_image_B[0]))

            self.truncateAndSave(None, real_images, synthetic_images, reconstructed_images,
                                 'images/{}/{}.png'.format(
                                     self.date_time, 'tmp'))
        except: # Ignore if file is open
            pass


    def print_ETA(self, start_time, epoch, epoch_iterations, loop_index):
        passed_time = time.time() - start_time

        iterations_so_far = ((epoch - 1) * epoch_iterations + loop_index) / self.batch_size
        iterations_total = self.epochs * epoch_iterations / self.batch_size
        iterations_left = iterations_total - iterations_so_far
        eta = round(passed_time / (iterations_so_far + 1e-5) * iterations_left)

        passed_time_string = str(datetime.timedelta(seconds=round(passed_time)))
        eta_string = str(datetime.timedelta(seconds=eta))
        print('Time passed', passed_time_string, ': ETA in', eta_string)

# Save and load
    def saveModel(self, model, epoch):
        # Create folder to save model architecture and weights
        directory = os.path.join('saved_models', self.date_time)
        if not os.path.exists(directory):
            os.makedirs(directory)

        model_path_w = 'saved_models/{}/{}_weights_epoch_{}.hdf5'.format(self.date_time, model.name, epoch)
        model.save_weights(model_path_w)
        model_path_m = 'saved_models/{}/{}_model_epoch_{}.json'.format(self.date_time, model.name, epoch)
        model.save_weights(model_path_m)
        json_string = model.to_json()
        with open(model_path_m, 'w') as outfile:
            json.dump(json_string, outfile)
        print('{} has been saved in saved_models/{}/'.format(model.name, self.date_time))

    def writeLossDataToFile(self, history):
        keys = sorted(history.keys())
        with open('images/{}/loss_output.csv'.format(self.date_time), 'w') as csv_file:
            writer = csv.writer(csv_file, delimiter=',')
            writer.writerow(keys)
            writer.writerows(zip(*[history[key] for key in keys]))

    def writeMetaDataToJSON(self):

        directory = os.path.join('images', self.date_time)
        if not os.path.exists(directory):
            os.makedirs(directory)
        # Save meta_data
        data = {}
        data['meta_data'] = []
        data['meta_data'].append({
            'img shape: height,width,channels': self.img_shape,
            'batch size': self.batch_size,
            'save interval': self.save_interval,
            'normalization function': str(self.normalization),
            'lambda': self.lambda_C,
            'lambda': self.lambda_C,
            'lambda_d': self.lambda_D,
            'learning_rate_D': self.learning_rate_D,
            'learning rate G': self.learning_rate_G,
            'epochs': self.epochs,
            'generator iterations': self.generator_iterations,
            'discriminator iterations': self.discriminator_iterations,
            'beta 1': self.beta_1,
            'beta 2': self.beta_2,
            'REAL_LABEL': self.REAL_LABEL,
            'number of A train examples': len(self.A_train),
            'number of B train examples': len(self.B_train),
            'number of A test examples': len(self.A_test),
            'number of B test examples': len(self.B_test),
        })

        with open('images/{}/meta_data.json'.format(self.date_time), 'w') as outfile:
            json.dump(data, outfile, sort_keys=True)

    def load_model_and_weights(self, model):
        path_to_model = os.path.join('generate_images', 'models', '{}.json'.format(model.name))
        path_to_weights = os.path.join('generate_images', 'models', '{}.hdf5'.format(model.name))
        #model = model_from_json(path_to_model)
        model.load_weights(path_to_weights)

    def load_model_and_generate_synthetic_images(self):
        response = input('Are you sure you want to generate synthetic images instead of training? (y/n): ')[0].lower()
        if response == 'y':
            self.load_model_and_weights(self.G_A2B)
            self.load_model_and_weights(self.G_B2A)
            synthetic_images_B = self.G_A2B.predict(self.A_test)
            synthetic_images_A = self.G_B2A.predict(self.B_test)

            def save_image(image, name, domain):
                if self.channels == 1:
                    image = image[:, :, 0]
                toimage(image, cmin=-1, cmax=1).save(os.path.join(
                    'generate_images', 'synthetic_images', domain, name))

            # Test A images
            for i in range(len(synthetic_images_A)):
                # Get the name from the image it was conditioned on
                name = self.testB_image_names[i].strip('.png') + '_synthetic.png'
                synt_A = synthetic_images_A[i]
                save_image(synt_A, name, 'A')

            # Test B images
            for i in range(len(synthetic_images_B)):
                # Get the name from the image it was conditioned on
                name = self.testA_image_names[i].strip('.png') + '_synthetic.png'
                synt_B = synthetic_images_B[i]
                save_image(synt_B, name, 'B')

            print('{} synthetic images have been generated and placed in ./generate_images/synthetic_images'
                  .format(len(self.A_test) + len(self.B_test)))


# https://stackoverflow.com/questions/50677544/reflection-padding-conv2d
class ReflectionPadding2D(Layer):
    def __init__(self, padding=(1, 1), **kwargs):
        self.padding = tuple(padding)
        self.input_spec = [InputSpec(ndim=4)]
        super(ReflectionPadding2D, self).__init__(**kwargs)

    def compute_output_shape(self, s):
        return (s[0], s[1] + 2 * self.padding[0], s[2] + 2 * self.padding[1], s[3])

    def call(self, x, mask=None):
        w_pad, h_pad = self.padding
        return tf.pad(x, [[0, 0], [h_pad, h_pad], [w_pad, w_pad], [0, 0]], 'REFLECT')


class ImagePool():
    def __init__(self, pool_size):
        self.pool_size = pool_size
        if self.pool_size > 0:
            self.num_imgs = 0
            self.images = []

    def query(self, images):
        if self.pool_size == 0:
            return images
        return_images = []
        for image in images:
            if len(image.shape) == 3:
                image = image[np.newaxis, :, :, :]

            if self.num_imgs < self.pool_size:  # fill up the image pool
                self.num_imgs = self.num_imgs + 1
                if len(self.images) == 0:
                    self.images = image
                else:
                    self.images = np.vstack((self.images, image))

                if len(return_images) == 0:
                    return_images = image
                else:
                    return_images = np.vstack((return_images, image))

            else:  # 50% chance that replace an old synthetic image
                p = random.uniform(0, 1)
                if p > 0.5:
                    random_id = random.randint(0, self.pool_size - 1)
                    tmp = self.images[random_id, :, :, :]
                    tmp = tmp[np.newaxis, :, :, :]
                    self.images[random_id, :, :, :] = image[0, :, :, :]
                    if len(return_images) == 0:
                        return_images = tmp
                    else:
                        return_images = np.vstack((return_images, tmp))
                else:
                    if len(return_images) == 0:
                        return_images = image
                    else:
                        return_images = np.vstack((return_images, image))

        return return_images



ModuleNotFoundError: No module named 'keras_contrib'

In [ ]:
GAN = CycleGAN()

In [ ]:
GAN.G_A2B.summary()

In [ ]:
test_D_model=GAN.Build_D()
test_D_model.summary()

In [ ]:
GAN.train(epochs=GAN.epochs, batch_size=GAN.batch_size, save_interval=GAN.save_interval)

In [ ]:
import matplotlib.pyplot as plt
if not os.path.exists('output/'):
    os.makedirs('output')
    
if not os.path.exists('output/trainA2B/'):
    os.makedirs('output/trainA2B')
    os.makedirs('output/testA2B')
    os.makedirs('output/trainB2A')
    os.makedirs('output/testB2A')
    
for i in range(0,len(GAN.A_train)):
    i_A=((GAN.A_train[i]+1)/2).reshape(1,256,256,3)
    i_B=(GAN.G_A2B.predict(i_A).reshape(256,256,3)+1)/2
    i_append=np.append(i_A.reshape(256,256,3),i_B,axis=1)
    plt.imsave("output/trainA2B/train%s"%i,i_append)
    
for i in range(0,len(GAN.A_test)):
    i_A=((GAN.A_test[i]+1)/2).reshape(1,256,256,3)
    i_B=(GAN.G_A2B.predict(i_A).reshape(256,256,3)+1)/2
    i_append=np.append(i_A.reshape(256,256,3),i_B,axis=1)
    plt.imsave("output/testA2B/test%s"%i,i_append)
    
for i in range(0,len(GAN.B_train)):
    i_B=((GAN.B_train[i]+1)/2).reshape(1,256,256,3)
    i_A=(GAN.G_B2A.predict(i_B).reshape(256,256,3)+1)/2
    i_append=np.append(i_B.reshape(256,256,3),i_A,axis=1)
    plt.imsave("output/trainB2A/train%s"%i,i_append)
    
for i in range(0,len(GAN.B_test)):
    i_B=((GAN.B_test[i]+1)/2).reshape(1,256,256,3)
    i_A=(GAN.G_B2A.predict(i_B).reshape(256,256,3)+1)/2
    i_append=np.append(i_B.reshape(256,256,3),i_A,axis=1)
    plt.imsave("output/testB2A/test%s"%i,i_append)